<a href="https://colab.research.google.com/github/madhumitha98/AI_Projects/blob/master/openAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv("lyrics_data.csv")

df.head()

,lyrics_filename,mood,lyrics,mood_cats
0,Five_Finger_Death_Punch___Bulletproof,anger,"""Bulletproof Lyrics[Verse 1]\nYou can take mon...",3
1,Karen_O_And_The_Kids___Hideaway,sad,"""Hideaway LyricsHideaway\nWell, they'll seat u...",1
2,ATB___These_Days,happy,"""These Days Lyrics[Verse 1]\nLet's take the lo...",0
3,Brian_Poole__The_Tremeloes___Do_You_Love_Me,happy,"""Main Street Chapter XXXI LyricsI\nTheir night...",0
4,Aretha_Franklin___Freeway_Of_Love,happy,"""Freeway of Love Lyrics[Verse 1]\nKnew you'd b...",0


In [ ]:
df.shape

(2539, 4)

In [ ]:
final_data = df.drop(columns=['lyrics_filename','mood_cats'],axis= 1)

final_data.columns = ['completion','prompt']

In [ ]:
final_data.to_json("train_lyrics.jsonl", orient="records", lines = True)


In [ ]:
!openai tools fine_tunes.prepare_data -f train_lyrics.jsonl -q

Analyzing...

- Your file contains 2539 prompt-completion pairs
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- There are 6 examples that are very long. These are rows: [3, 1294, 1508, 1720, 2352, 2528]
For conditional generation, and for classification the examples shouldn't be longer than 2048 tokens.
- All prompts end with suffix `, 'default'`. This suffix seems very long. Consider replacing with a shorter suffix, such as ` ->`
- The completion should start with a whitespace character (` `). This tends to produce better results due to the tokenization we use. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details

Based on the analysis we will perform the following actions:
- [

In [ ]:
import openai
from dotenv import load_dotenv
load_dotenv()

import os
openai.api_key = os.getenv('OPENAI_API_KEY')


In [ ]:
train_file = openai.files.create(file=open("train_lyrics_prepared_train.jsonl", "rb"), purpose="fine-tune")
valid_file = openai.files.create(file=open("train_lyrics_prepared_valid.jsonl", "rb"), purpose="fine-tune")

In [ ]:
fine_tuning_job = openai.fine_tuning.jobs.create(training_file=train_file.id, validation_file=valid_file.id, model="babbage-002")

print(fine_tuning_job)

FineTuningJob(id='ftjob-DeOKPQUuGtlWZg7kSu58unJf', created_at=1708555064, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='babbage-002', object='fine_tuning.job', organization_id='org-X4fybrNrNcQQX82ayQW0sbv0', result_files=[], status='validating_files', trained_tokens=None, training_file='file-1Sk7oTnro4kjSyUj8Qroupiw', validation_file='file-Ogz4cJFgy2GB1mWoyY8mkvGr')
